# install packages
install packages need to train mario agent.

In [ ]:
!pip install torch
!pip install numpy
!pip install matplotlib
!pip install gymnasium==0.29.1
!pip install gym-super-mario-bros==7.4.0
!pip install gym==0.25.2
!pip install imageio-ffmpeg
!pip install imageio
!pip install torchvision
!pip install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.3 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=535717 sha256=c4a64388a13d2628d1c96581067805a9c607c00580c59f435d8e308c60adf154
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (7

# import packages

In [ ]:
from PIL import Image
from collections import deque
from datetime import datetime
from pathlib import Path
import copy
import cv2
import imageio
import numpy as np
import random, os
import torch
from torch import nn
import torch.nn.functional as F
import torch.multiprocessing as mp
#import multiprocessing as mp
from torchvision import transforms as T

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY

# Create hyperparammeters
Config hyperparammeters, just change it

In [ ]:
#class DictWrapper created by Chatgpt
class DictWrapper:
    def __init__(self, dictionary):
        self._dict = dictionary

    def __getattr__(self, item):
        if item in self._dict:
            return self._dict[item]
        else:
            raise AttributeError(f"'DictWrapper' object has no attribute '{item}'")

config = {
    'num_envs': 16,
    'save_model_step': int(1e5),
    'save_figure_step': int(1e3),
    'learn_step': 20,
    'total_step_or_episode': 'step',
    'total_step': int(5e6),
    'total_episode': None,
    'save_dir': "",
    'gamma': 0.9,
    'learning_rate': 1e-4,
    'state_dim': (1, 84, 84),
    'action_dim': 12,#12 for complex, 7 for simple
    'entropy_coef': 0.01,
    'V_coef': 0.5,
    'max_grad_norm': 0.5,
    'world': 1,
    'stage': 1,
    'action_type': 'complex'
}

config = DictWrapper(config)

In [ ]:
gym.__version__

'0.25.2'

# Define environment
## Create a custom environment, We need:
- SkipFrame: because the episode is very long then in this environment, we only need to repeat action sometimes. Then we repeat each action 4 times and skip 3 first frames (return 4th frame). We need to sum 4 rewards from 4 frames.
- GrayScaleResizeObservation: grayscale state (from RGB to gray image) and resize to 84x84.
- NoopResetEnv: when resetting the environment, I do random actions sometimes before using the evironment. This same as atari strategy. When reset, we choice num_noops random actions by random from 0 to noop_max and random do num_noops actions. If random actions yield to a terminal state, then reset and continue do random action. I set noop_max as 30 same as atari.
- CustomRewardAndDoneEnv: I see that a lot of people train Mario to use this custom reward, and then I copy it. Just add 50 rewards if the agent solves the stage and add -50 rewards if the agent is dead. And the reward is divided into 10. I set done to True if Mario dies instead of when Mario lost all lives as default.
- stage 4-2: give a -50 reward when Mario moves on top of the map (y_pos >= 255).
- with stages 4-4 and 7-4: set done = True when Mario goes to wrong way and gives a -50 penalty reward. If Mario goes to the true way but the map still loops (this is a bug), I set done = True but don't give a penalty reward.

## About reward system:
- set done to True when Mario dies: This is the most important thing because, in the default reward system, Mario still yields a reward by just moving right, if Mario dies, the agent can't lose total rewards and the agent still go right (in new life) and get more rewards. This is the easiest way to get rewards and the agent also learns this trick.
- penalty -50 reward when Mario dies: this is necessary to make Mario train faster. If we don't add a penalty, the agent can't complete hard stages.
reward 50 when reaching the flag: this makes Mario train faster and surpass stuck points in hard stages.
- change penalty and reaching flag reward of more or less than 50 don't make more difference then I don't change it.
- Divide rewards by 10: I think it makes total rewards smaller and the agent can learn a better way, but I am not sure how it is necessary. I just copy it.
- With Stage 4-2: I see that the agent can reach more rewards when Mario goes to the wrap zone, but Mario can't win this stage with the wrap zone way because the reward system gives negative rewards when Mario goes left. Then I just give a penalty reward when Mario moves to the top of the map.
- with stage 4-4 and 7-4:
+ because this map has the wrong way, Mario will go to the loop and the reward still increase to infinity, then I need to set done = True and give a negative penalty reward
+ negative reward to prevent Mario go to the wrong way.
+ Another strategy is to give a negative reward but don't set done = True (like 4-2). But this map has a bug and this strangtegy not working.
+ even though Mario goes in the correct way, sometimes Mario still goes in the loop. Then I set done = True every time Mario goes on the loop (check by x_pos and max_x_pos).

In [ ]:
# Initialize Super Mario environment (in v0.26 change render mode to 'human' to see results on the screen)
if gym.__version__ < '0.26':
    env = gym_super_mario_bros.make(f"SuperMarioBros-{config.world}-{config.stage}-v0", new_step_api=True)
else:
    env = gym_super_mario_bros.make(f"SuperMarioBros-{config.world}-{config.stage}-v0", render_mode='rgb', apply_api_compatibility=True)

env = JoypadSpace(env, COMPLEX_MOVEMENT)
print(env.action_space)

env.reset()
next_state, reward, done, trunc, info = env.step(action=0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, trunk, info

class GrayScaleResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        self.current_state = observation
        observation = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        observation = cv2.resize(observation, self.shape, interpolation=cv2.INTER_AREA)
        observation = observation.astype(np.uint8).reshape(-1, observation.shape[0], observation.shape[1])
        return observation

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        super(NoopResetEnv, self).__init__(env)
        self.noop_max = noop_max

    def reset(self, **kwargs):
        """Do no-op action for a number of steps in [1, noop_max]."""
        obs = self.env.reset(**kwargs)
        noops = np.random.randint(0, self.noop_max, (1, ))[0]
        for _ in range(noops):
            action = self.env.action_space.sample()
            obs, _, done, _, _ = self.env.step(action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        obs, reward, done, trunk, info = self.env.step(ac)
        return obs, reward, done, trunk, info

class CustomRewardAndDoneEnv(gym.Wrapper):
    def __init__(self, env=None, world=1, stage=1):
        super(CustomRewardAndDoneEnv, self).__init__(env)
        self.current_score = 0
        self.current_x = 0
        self.current_x_count = 0
        self.max_x = 0
        self.world = world
        self.stage = stage

    def reset(self, **kwargs):
        self.current_score = 0
        self.current_x = 0
        self.current_x_count = 0
        self.max_x = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, done, trunc, info = self.env.step(action)
        if (info['x_pos'] - self.current_x) == 0:
            self.current_x_count += 1
        else:
            self.current_x_count = 0
        if info["flag_get"]:
            reward += 50
            done = True
        if done and info["flag_get"] == False and info["time"] != 0:
            reward -= 50
            done = True
        self.current_score = info["score"]
        self.current_x = info["x_pos"]
        self.max_x = max(self.max_x, self.current_x)

        if self.world == 7 and self.stage == 4:
            if (506 <= info["x_pos"] <= 832 and info["y_pos"] > 127) or (
                    832 < info["x_pos"] <= 1064 and info["y_pos"] < 80) or (
                    1113 < info["x_pos"] <= 1464 and info["y_pos"] < 191) or (
                    1579 < info["x_pos"] <= 1943 and info["y_pos"] < 191) or (
                    1946 < info["x_pos"] <= 1964 and info["y_pos"] >= 191) or (
                    1984 < info["x_pos"] <= 2060 and (info["y_pos"] >= 191 or info["y_pos"] < 127)) or (
                    2114 < info["x_pos"] < 2440 and info["y_pos"] < 191):
                reward -= 50
                done = True
            if done == False and info["x_pos"] < self.max_x - 100:
                done = True
        if self.world == 4 and self.stage == 4:
            if (info["x_pos"] <= 1500 and info["y_pos"] < 127) or (
                    1588 <= info["x_pos"] < 2380 and info["y_pos"] >= 127):
                reward = -50
                done = True
            if done == False and info["x_pos"] < self.max_x - 100:
                done = True
            if done == False:
                reward -= 0.1
        if self.world == 4 and self.stage == 2 and done == False and info['y_pos'] >= 255:
            reward -= 50

        return state, reward / 10., done, trunc, info

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


Discrete(12)
(240, 256, 3),
 0.0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


# Create MultipleEnvironments
MultipleEnvironments use multi-processing to parallel running.

Because in the training process, we need to reset the environment when the agent reaches the terminal state. But if we will do it in parallel, then I don't want to check each environment and reset (by loop) or create a new function that parallels check and reset all environments. Then I reset the environment if done = True in step function and set next_state = env.reset(). Then in training, we just set state = next_state (next_state is reset state if done = True)

In [ ]:
#modify from https://github.com/uvipen/Super-mario-bros-PPO-pytorch/blob/master/src/env.py
def create_env(world, stage, action_type, test=False):
    if gym.__version__ < '0.26':
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v0", new_step_api=True)
    else:
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v0", render_mode='rgb', apply_api_compatibility=True)

    if action_type == "right":
        action_type = RIGHT_ONLY
    elif action_type == "simple":
        action_type = SIMPLE_MOVEMENT
    else:
        action_type = COMPLEX_MOVEMENT

    env = JoypadSpace(env, action_type)

    if test == False:
        env = NoopResetEnv(env)
    env = SkipFrame(env, skip=4)
    env = CustomRewardAndDoneEnv(env, world, stage)
    env = GrayScaleResizeObservation(env, shape=84)
    # if gym.__version__ < '0.26':
    #     env = FrameStack(env, num_stack=4, new_step_api=True)
    # else:
    #     env = FrameStack(env, num_stack=4)
    return env

class MultipleEnvironments:
    def __init__(self, world, stage, action_type, num_envs):
        self.agent_conns, self.env_conns = zip(*[mp.Pipe(duplex=True) for _ in range(num_envs)])
        self.envs = [create_env(world, stage, action_type) for _ in range(num_envs)]

        for index in range(num_envs):
            process = mp.Process(target=self.run, args=(index,))
            process.start()
            self.env_conns[index].close()

    def run(self, index):
        self.agent_conns[index].close()
        while True:
            request, action = self.env_conns[index].recv()
            if request == "step":
                next_state, reward, done, trunc, info = self.envs[index].step(action)
                if done:
                    next_state = self.envs[index].reset()
                self.env_conns[index].send((next_state, reward, done, trunc, info))
            elif request == "reset":
                self.env_conns[index].send(self.envs[index].reset())
            else:
                raise NotImplementedError

    def step(self, actions):
        [agent_conn.send(("step", act)) for agent_conn, act in zip(self.agent_conns, actions)]
        next_states, rewards, dones, truncs, infos = zip(*[agent_conn.recv() for agent_conn in self.agent_conns])
        return next_states, rewards, dones, truncs, infos

    def reset(self):
        [agent_conn.send(("reset", None)) for agent_conn in self.agent_conns]
        states = [agent_conn.recv() for agent_conn in self.agent_conns]
        return states

In [ ]:
mp.cpu_count()

2

# Create memory
Memory just save all info we need to train and return all stored info

In [ ]:
class Memory():
    def __init__(self, num_envs):
        self.num_envs = num_envs

        self.states = []
        self.actions = []
        self.next_states = []
        self.rewards = []
        self.dones = []
        self.logits = []
        self.values = []

    def save(self, state, action, reward, next_state, done, logit, V):
        self.states.append(state)
        self.actions.append(action)
        self.next_states.append(next_state)
        self.rewards.append(reward)
        self.dones.append(done)
        self.logits.append(logit)
        self.values.append(V)

    def reset(self):
        self.states = []
        self.actions = []
        self.next_states = []
        self.rewards = []
        self.dones = []
        self.logits = []
        self.values = []

    def get_data(self):
        return self.states, self.actions, self.next_states, self.rewards, self.dones, self.logits, self.values

# Create agent
The agent includes 4 main functions:
## train
train function train agent via many episodes:
- first, we create (h, c) as zeros. Reset the first state.
- loop until the agent wins this stage or reaches the maximum episode/step:
- predict value, logit, h, c for the current state
- sample action from logit with category distribution (select_action function)
- log all info to memory
- train agent every learn_step (learn function)
- eval agent every save_figure_step (save_figure function)
- reset (h, c) to zeros if the agent reaches the terminal state
- set state = next_state (I reset environment when agent reach terminal state then next_state is first state if done=True)

## select_action
this function sample action from logit:
- just convert logit to probability: policy = F.softmax(logits, dim=1)
- create distribution from probability: distribution = torch.distributions.Categorical(policy)
- sample action from distribution: actions = distribution.sample()

## save_figure
this function eval agent and saves agent/video if the agent yields better total rewards:
- create (h, c) as zeros, and reset the environment.
- loop until the agent reaches the terminal state.
- predict logit from model
- get action = argmax (logit)
- environment do this action to get next_state, reward, info, done
- if total_reward > best test total reward or agent complete this stage, I save model and video.
- if agent completes this state, we stop training.

## learn
this function trains agent from the experiment saved in memory
- get all the info from memory
- calculate td (lambda) target and gae advantages
- calculate loss
- norm gradient
- update model from loss

In [ ]:
class Agent():
    def __init__(self, world, stage, action_type, envs, num_envs, state_dim, action_dim, save_dir, save_model_step,
                 save_figure_step, learn_step, total_step_or_episode, total_step, total_episode, model,
                 gamma, learning_rate, entropy_coef, V_coef, max_grad_norm, device):
        self.world = world
        self.stage = stage
        self.action_type = action_type

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir
        self.learn_step = learn_step
        self.total_step_or_episode = total_step_or_episode
        self.total_step = total_step
        self.total_episode = total_episode

        self.current_step = 0
        self.current_episode = 0

        self.save_model_step = save_model_step
        self.save_figure_step = save_figure_step

        self.device = device
        self.save_dir = save_dir

        self.num_envs = num_envs
        self.envs = envs
        self.model = model.to(self.device)

        self.learning_rate = learning_rate
        self.gamma = gamma
        self.entropy_coef = entropy_coef
        self.V_coef = V_coef
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        self.max_grad_norm = max_grad_norm

        self.memory = Memory(self.num_envs)

        self.is_completed = False

        self.env = None
        self.max_test_score = -1e9

        # I just log 1000 lastest update and print it to log.
        self.V_loss = np.zeros((1000,)).reshape(-1)
        self.P_loss = np.zeros((1000,)).reshape(-1)
        self.E_loss = np.zeros((1000,)).reshape(-1)
        self.total_loss = np.zeros((1000,)).reshape(-1)
        self.loss_index = 0
        self.len_loss = 0

    def save_figure(self, is_training=False):
        # test current model and save model/figure if model yield best total rewards.
        # create env for testing, reset test env
        if self.env is None:
            self.env = create_env(self.world, self.stage, self.action_type, True)
        state = self.env.reset()
        done = False

        images = []
        total_reward = 0
        total_step = 0
        num_repeat_action = 0
        old_action = -1

        # create h, c as zeros
        h = torch.zeros((1, 512), dtype=torch.float, device = self.device)
        c = torch.zeros((1, 512), dtype=torch.float, device = self.device)

        episode_time = datetime.now()

        # play 1 episode, just get loop action with max probability from model until the episode end.
        while not done:
            with torch.no_grad():
                logit, V, h, c = self.model(torch.tensor(state, dtype = torch.float, device = self.device).unsqueeze(0), h, c)
            action = logit.argmax(-1).item()
            next_state, reward, done, trunc, info = self.env.step(action)
            state = next_state
            img = Image.fromarray(self.env.current_state)
            images.append(img)
            total_reward += reward
            total_step += 1

            if action == old_action:
                num_repeat_action += 1
            else:
                num_repeat_action = 0
            old_action = action
            if num_repeat_action == 200:
                break

        #logging, if model yield better result, save figure (test_episode.mp4) and model (best_model.pth)
        if is_training:
            f_out = open(f"logging_test.txt", "a")
            f_out.write(f'episode_reward: {total_reward} episode_step: {total_step} current_step: {self.current_step} loss_p: {(self.P_loss.sum()/self.len_loss):.4f} loss_v: {(self.V_loss.sum()/self.len_loss):.4f} loss_e: {(self.E_loss.sum()/self.len_loss):.4f} loss: {(self.total_loss.sum()/self.len_loss):.4f} episode_time: {datetime.now() - episode_time}\n')
            f_out.close()

        if total_reward > self.max_test_score or info['flag_get']:
            imageio.mimsave('test_episode.mp4', images)
            self.max_test_score = total_reward
            if is_training:
                torch.save(self.model.state_dict(), f"best_model.pth")

        # if model can complete this game, stop training by set self.is_completed to True
        if info['flag_get']:
            self.is_completed = True

    def save_model(self):
        torch.save(self.model.state_dict(), f"model_{self.current_step}.pth")

    def load_model(self, model_path = None):
        if model_path is None:
            model_path = f"model_{self.current_step}.pth"
        self.model.load_state_dict(torch.load(model_path))

    def select_action(self, states, h, c):
        # select action when training, we need use Categorical distribution to make action base on probability from model
        states = torch.tensor(np.array(states), device = self.device)

        logits, V, h, c = self.model(states, h, c)
        with torch.no_grad():
            policy = F.softmax(logits, dim=1)
            distribution = torch.distributions.Categorical(policy)
            actions = distribution.sample().cpu().numpy().tolist()
        return actions, logits, V, h, c

    def update_loss_statis(self, loss_p, loss_v, loss_e, loss):
        # update loss for logging, just save 1000 latest updates.
        self.V_loss[self.loss_index] = loss_v
        self.P_loss[self.loss_index] = loss_p
        self.E_loss[self.loss_index] = loss_e
        self.total_loss[self.loss_index] = loss
        self.loss_index = (self.loss_index + 1)%1000
        self.len_loss = min(self.len_loss+1, 1000)

    def learn(self, h, c):
        # reset optimizer
        self.optimizer.zero_grad()

        # get all data
        states, actions, next_states, rewards, dones, logits, values = self.memory.get_data()

        # calculate target (td lambda target) and gae advantages
        targets = []
        with torch.no_grad():
            _, next_value, h, c = self.model(torch.tensor(np.array(next_states[-1]), device = self.device), h, c)
        target = next_value
        advantage = 0

        for state, next_state, reward, done, V in zip(states[::-1], next_states[::-1], rewards[::-1], dones[::-1], values[::-1]):
            done = torch.tensor(done, device = self.device, dtype = torch.float).reshape(-1, 1)
            reward = torch.tensor(reward, device = self.device).reshape(-1, 1)

            target = next_value * self.gamma * (1-done) + reward
            advantage = target + self.gamma * advantage * (1-done)
            targets.append(advantage)
            advantage = advantage - V.detach()
            next_value = V.detach()
        targets = targets[::-1]

        # convert all data to tensor
        values = torch.cat(values, 0)
        targets = torch.cat(targets, 0).view(-1, 1)
        logits = torch.cat(logits, 0)
        probs = torch.softmax(logits, -1)
        advantages = (targets - values).reshape(-1)

        # calculate loss
        entropy = (- (probs * (probs + 1e-9).log()).sum(-1)).mean()
        loss_V = F.smooth_l1_loss(values, targets)

        index = torch.arange(0, len(probs), device = self.device)
        actions = torch.flatten(torch.tensor(actions, device = self.device, dtype = torch.int64))
        loss_P = -((probs[index, actions] + 1e-9).log() * advantages.detach()).mean()

        loss = - entropy * self.entropy_coef + loss_V * self.V_coef + loss_P
        loss.backward()

        # norm gradient and update agent
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)

        self.optimizer.step()

        self.update_loss_statis(loss_P.item(), loss_V.item(), entropy.item(), loss.item())

    def train(self):
        episode_reward = [0] * self.num_envs
        episode_step = [0] * self.num_envs
        max_episode_reward = 0
        max_episode_step = 0
        episode_time = [datetime.now() for _ in range(self.num_envs)]
        total_time = datetime.now()

        last_episode_rewards = []

        # reset envs
        states = self.envs.reset()

        # create h, c as zeros
        h = torch.zeros((self.num_envs, 512), dtype=torch.float, device = self.device)
        c = torch.zeros((self.num_envs, 512), dtype=torch.float, device = self.device)

        while True:
            # finish training if agent reach total_step or total_episode based on what type of total_step_or_episode is step or episode
            self.current_step += 1

            if self.total_step_or_episode == 'step':
                if self.current_step >= self.total_step:
                    break
            else:
                if self.current_episode >= self.total_episode:
                    break

            actions, logit, V, h, c = self.select_action(states, h, c)

            next_states, rewards, dones, truncs, infos = self.envs.step(actions)

            # save to memory
            self.memory.save(states, actions, rewards, next_states, dones, logit, V)

            episode_reward = [x + reward for x, reward in zip(episode_reward, rewards)]
            episode_step = [x+1 for x in episode_step]

            # logging after each step, if 1 episode is ending, I will log this to logging.txt
            for i, done in enumerate(dones):
                if done:
                    self.current_episode += 1
                    max_episode_reward = max(max_episode_reward, episode_reward[i])
                    max_episode_step = max(max_episode_step, episode_step[i])
                    last_episode_rewards.append(episode_reward[i])
                    f_out = open(f"logging.txt", "a")
                    f_out.write(f'episode: {self.current_episode} agent: {i} rewards: {episode_reward[i]:.4f} steps: {episode_step[i]} complete: {infos[i]["flag_get"]==True} mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean():.4f} max_rewards: {max_episode_reward:.4f} max_steps: {max_episode_step} current_step: {self.current_step} loss_p: {(self.P_loss.sum()/self.len_loss):.4f} loss_v: {(self.V_loss.sum()/self.len_loss):.4f} loss_e: {(self.E_loss.sum()/self.len_loss):.4f} loss: {(self.total_loss.sum()/self.len_loss):.4f} episode_time: {datetime.now() - episode_time[i]} total_time: {datetime.now() - total_time}\n')
                    f_out.close()
                    episode_reward[i] = 0
                    episode_step[i] = 0
                    episode_time[i] = datetime.now()

            # reset h and c to zeros for enviroments that just ending episode, just multiply h and c with (1-dones)
            h = h * (1 - torch.tensor(dones, device = self.device, dtype = torch.float).reshape(-1, 1))
            c = c * (1 - torch.tensor(dones, device = self.device, dtype = torch.float).reshape(-1, 1))

            # training agent every learn_step
            if self.current_step % self.learn_step == 0:
                self.learn(h, c)
                self.memory.reset()

            if self.current_step % self.save_model_step == 0:
                self.save_model()

            # eval agent every save_figure_step
            if self.current_step % self.save_figure_step == 0 and self.save_figure_step != -1:
                self.save_figure(is_training=True)
                if self.is_completed:
                    f_out = open(f"logging.txt", "a")
                    f_out.write(f' mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean()} max_rewards: {max_episode_reward} max_steps: {max_episode_step} current_step: {self.current_step} total_time: {datetime.now() - total_time}\n')
                    f_out.close()
                    return

            states = list(next_states)

        f_out = open(f"logging.txt", "a")
        f_out.write(f' mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean()} max_rewards: {max_episode_reward} max_steps: {max_episode_step} current_step: {self.current_step} total_time: {datetime.now() - total_time}\n')
        f_out.close()

# Create model
I use the same model architecture as [github/uvipen/Super-mario-bros-A3C-pytorch/model](https://github.com/uvipen/Super-mario-bros-A3C-pytorch/blob/master/src/model.py).
Model includes:
- 4 convolution layers to encode input image (observation) to feature vector.
- LSTM Cell to capture past observations.
- two linear layers for policy and value prediction (actor and critic)

To understand how LSTM (recurrent network) works in RL, we can view (h, c) from the lastest step as input of the model. Then the model takes 3 inputs: (state, h, c) instead of just state as a non-recurrent model. Then we just save output (h, c) for each state and reuse it as input with (next_state, h, c).

After completing this project, I just tried to remove _initialize_weights and performance did not change in some stages, then I thought _initialize_weights is not necessary, but I don't want to remove it because I am not sure this code works for all stages without _initialize_weights

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(input_dim[0], 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.lstm = nn.LSTMCell(1152, 512)
        self.critic_linear = nn.Linear(512, 1)
        self.actor_linear = nn.Linear(512, output_dim)
        self._initialize_weights()

    def _initialize_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                # nn.init.kaiming_uniform_(module.weight)
                nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.LSTMCell):
                nn.init.constant_(module.bias_ih, 0)
                nn.init.constant_(module.bias_hh, 0)

    def forward(self, x, h, c):
        x = F.relu(self.conv1(x/255.))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        h, c = self.lstm(x, (h, c))
        return self.actor_linear(h), self.critic_linear(h), h.detach(), c.detach()

model = Model(config.state_dim, config.action_dim)

# train

In [ ]:
envs = MultipleEnvironments(config.world, config.stage, config.action_type, config.num_envs)

In [ ]:
envs.envs[0]

<GrayScaleResizeObservation<CustomRewardAndDoneEnv<SkipFrame<NoopResetEnv<JoypadSpace<TimeLimit<OrderEnforcing<StepAPICompatibility<PassiveEnvChecker<SuperMarioBrosEnv<SuperMarioBros-1-1-v0>>>>>>>>>>>

In [ ]:
agent = Agent(world = config.world, stage = config.stage, action_type = config.action_type, envs = envs, num_envs = config.num_envs, 
              state_dim = config.state_dim, action_dim = config.action_dim, save_dir = config.save_dir,
              save_model_step = config.save_model_step, save_figure_step = config.save_figure_step, learn_step = config.learn_step,
              total_step_or_episode = config.total_step_or_episode, total_step = config.total_step, total_episode = config.total_episode,
              model = model, gamma = config.gamma, learning_rate = config.learning_rate, entropy_coef = config.entropy_coef, V_coef = config.V_coef,
              max_grad_norm = config.max_grad_norm,
              device = "cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
agent.train()

# test

In [ ]:
agent.load_model("best_model.pth")
agent.save_figure()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
